In [1]:
import wandb 
import random
import math

config = {'learning_rate': 0.02, "architecture": "CNN", "dataset": "CIFAR-100", "epochs": 10}

wandb.init(project="meta-labelling", config=config)

epochs = 10
offset = random.random() / 5 
for epoch in range(2, epochs):
    acc = 0.5 * (math.sin(epoch / 2 + offset) + 1)
    loss = 1 - acc + random.random() / 10
    wandb.log({"acc": acc, "loss": loss})

wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /Users/lnshuti/.netrc


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


acc,███▆▅▃▂▁
loss,▁▁▂▃▄▆██
acc,0.00395
loss,1.04621
